In [1]:
!pip install --quiet git+https://github.com/CSIRO-enviro-informatics/loci-scripts.git@master
!pip install --quiet numpy pandas
!pip install --quiet seaborn

# Reapportioning ASGS16 LGA to SA2  example using the pyloci API

This notebook show how you would perform reapportioning using the [pyloci](https://pypi.org/project/pyloci/) library using a simple CSV file. The pyloci library interfaces directly with the SPARQL API for the Loc-I Cache GraphDB.

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('covid19-20200713.csv', delimiter = ',')
#show the csv file read in
df

,LGA,LGA ID,Confirmed cases (ever),Active cases (current)
0,HUME,http://linked.data.gov.au/dataset/asgs2016/loc...,416,222
1,MELBOURNE,http://linked.data.gov.au/dataset/asgs2016/loc...,312,217
2,WYNDHAM,http://linked.data.gov.au/dataset/asgs2016/loc...,306,209
3,BRIMBANK,http://linked.data.gov.au/dataset/asgs2016/loc...,271,127
4,MOONEE VALLEY,http://linked.data.gov.au/dataset/asgs2016/loc...,271,178
5,MORELAND,http://linked.data.gov.au/dataset/asgs2016/loc...,200,88
6,CASEY,http://linked.data.gov.au/dataset/asgs2016/loc...,144,23
7,BANYULE,http://linked.data.gov.au/dataset/asgs2016/loc...,140,42
8,WHITTLESEA,http://linked.data.gov.au/dataset/asgs2016/loc...,137,64


### Visualise LGA data

In [3]:
#visualise LGA 
import json
import requests
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = "https://api.loci.cat/api/v1/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    geojson_data = []
    if len( res['geometry']) > 0:
        geojson_data = res['geometry'][0]
    return geojson_data

In [4]:
import seaborn as sns
from ipywidgets import Text, HTML
from ipyleaflet import (WidgetControl)
import math
target_col = "Active cases (current)"
palette = sns.light_palette("navy", 3, reverse=True)
hexpal = palette.as_hex()
df['quantiles'] = pd.qcut(df[target_col], q=3, precision=0)
df['hex'] = pd.qcut(df[target_col], q=3, precision=0, labels=hexpal)


In [5]:
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = -37.8136
y_coord = 144.9631

map1 = ipy.Map(center=[x_coord, y_coord], zoom=9)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

html = HTML('''Hover over a region''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
map1.add_control(control)

def update_html(feature, **kwargs):
     html.value = '''
     <h4>LGA : {}</h4>
     <h4>No. Cases: {}</h4>
      '''.format(feature['id'], math.floor(feature['num_cases']))

res_df_cols = {
    "LGA" : [],
    "SA2" : [],
    "percent_overlap": [],
    target_col: [],
    "reapportioned_data": [],
}
for index, row in df.iterrows():
    fromFeature = row['LGA ID']
    value  = row[target_col]
    geojson_data = get_geom(fromFeature)
    geojson_data['num_cases'] = value
    curr_geojson_layer = GeoJSON(data=geojson_data, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': row['hex'], 
                          'fillOpacity': 0.7},
                     hover_style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': row['hex'], 
                          'fillOpacity': 0.9,                    
                    })
    curr_geojson_layer.on_hover(update_html)
    map1.add_layer(curr_geojson_layer)

    

In [6]:
map1

Map(center=[-37.8136, 144.9631], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

### Query LOC-I APIs to get SA2 regions for each LGA input

In [ ]:
%%time
from tqdm.notebook import tqdm

target_col = "Active cases (current)"
res_df_cols = {
    "LGA ID" : [],
    "SA2" : [],
    "percent_overlap": [],
    target_col: [],
    "reapportioned_data": [],
}
with tqdm(total=len(list(df.iterrows()))) as pbar:
    for index, row in df.iterrows():
        fromFeature = row['LGA ID']
        #print("{}, {}".format(row['LGA'], str(row[target_col])))
        #print("Querying overlaps of {} to {}...".format(fromFeature, toFeatureType))
        tic = time.perf_counter()
        list_locations = api_util.query_api_location_overlaps(fromFeature, toFeatureType, LOCI_INTEGRATION_API, crosswalk='true')
        toc = time.perf_counter()
        #print(f"query_api_location_overlaps took {toc - tic:0.4f} seconds")
        #print(list_locations)
        for o in list_locations['overlaps']:
            res_df_cols["LGA ID"].append(fromFeature)
            res_df_cols["SA2"].append(o['uri'])
            res_df_cols["percent_overlap"].append(o['forwardPercentage'])
            res_df_cols[target_col].append(row[target_col])
            reapportioned = (float(o['forwardPercentage'])/100.0)*float(row[target_col])
            res_df_cols["reapportioned_data"].append(reapportioned)
        pbar.update(1)

  

In [8]:
#sure the config for the SPARQL endpoint to hit is set
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
#print(os.getenv("SPARQL_ENDPOINT"))
#import the reapportioning module from pyloci
from pyloci import reapportioning
from pyloci.api.util import Util as API_Util
from pyloci.sparql import util as sparql_util
import time

auth = None
api_util = API_Util()
toFeatureType = "http://linked.data.gov.au/def/asgs#StatisticalAreaLevel2"
LOCI_INTEGRATION_API = "https://api.loci.cat/api/v1"


### Show results of the reapportioned data to SA2

In [9]:
res_df = pd.DataFrame (res_df_cols, columns = ['LGA ID', "SA2", "percent_overlap", target_col, "reapportioned_data"])
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_colwidth', 0)
res_df


,LGA ID,SA2,percent_overlap,Active cases (current),reapportioned_data


In [10]:
palette = sns.color_palette("Blues", 10)
hexpal = palette.as_hex()
res_df['quantiles'] = pd.qcut(res_df['reapportioned_data'], q=10, precision=0)
res_df['hex'] = pd.qcut(res_df['reapportioned_data'], q=10, precision=0, labels=hexpal)

#interval_range = pd.interval_range(start=0, freq=10000, end=200000)
#res_df['defined_ranges'] = pd.cut(res_df['reapportioned_data'], bins=interval_range, labels=[1,2,3])
#res_df.head()

ValueError: Bin edges must be unique: array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [ ]:
column = res_df["reapportioned_data"]
max_value = column.max()
interval_range = pd.interval_range(start=0, freq=10, end=max_value)
res_df['defined_ranges'] = pd.cut(res_df['reapportioned_data'], bins=interval_range)
#len(interval_range)

### Categorise reapportioned data into 10 quantiles 

In [ ]:
res_df

### Plot the SA2 reapportioned data on a map

In [ ]:
map3 = ipy.Map(center=[x_coord, y_coord], zoom=9)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

column = res_df["reapportioned_data"]
max_value = column.max()
min_value = column.min()

html2 = HTML('''Hover over a region''')
html2.layout.margin = '0px 20px 20px 20px'
control2 = WidgetControl(widget=html2, position='topright')
map3.add_control(control2)

def update_html2(feature, **kwargs):
     html2.value = '''
     <h4>ASGS SA2 ID: {}</h4>
     <h4>No. Cases: {}</h4>
      '''.format(feature['id'], math.floor(feature['num_cases']))

for index, row in res_df.iterrows():
    fromFeature = row['SA2']
    value = row['reapportioned_data']
    fillcolor = row['hex']
    geojson_data = get_geom(fromFeature)
    geojson_data['num_cases'] = value
    curr_geojson_layer = GeoJSON(data=geojson_data, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': fillcolor, 
                          'fillOpacity': 0.7,
                    },
                    hover_style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': fillcolor, 
                          'fillOpacity': 0.9,                    
                    },)
    curr_geojson_layer.on_hover(update_html2)
    map3.add_layer(curr_geojson_layer)

In [ ]:
map3